In [1]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
X, y = make_classification(n_samples=10000, weights=[0.7, 0.3], n_informative=5, n_features=10,
                           n_redundant=5, random_state=1)
X = pd.DataFrame(X)
y = pd.Series(y, name='target')
y.value_counts()

0    6982
1    3018
Name: target, dtype: int64

In [3]:
df = pd.concat([X, y], axis=1)
target = 'target'

## Балансировка данных

**Способы борьбы с дисбалансом классов**

1. Собрать больше данных
2. Выбрать подходящую метрику качества
3. Попробовать разные модели, одни модели более устойчивы к несбалансированным данным, чем другие
4. Штраф за ошибки при прогнозе меньшего класса
5. Undersampling и Oversampling

https://habr.com/ru/post/461285/
<img src="images/balancing.png">
Undersampling с использованием Tomek Links:
<img src="images/tomek.png">
Oversampling со SMOTE:
<img src="images/smote.png">

In [4]:
df[target].value_counts()

0    6982
1    3018
Name: target, dtype: int64

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2, stratify=y)

display(y_train.value_counts(normalize=True), y_test.value_counts(normalize=True))

0    0.69825
1    0.30175
Name: target, dtype: float64

0    0.698
1    0.302
Name: target, dtype: float64

In [6]:
disbalance = y_train.value_counts()[0] / y_train.value_counts()[1]
disbalance

2.3140016570008286

In [7]:
lr = LogisticRegressionCV()
lr.fit(X_train, y_train)

pred_train = lr.predict(X_train)
pred_test = lr.predict(X_test)

print(f'F1-мера на первом классе на трейне {round(f1_score(y_train, pred_train), 2)}')
print(f'F1-мера на первом классе на тесте {round(f1_score(y_test, pred_test), 2)}')

F1-мера на первом классе на трейне 0.68
F1-мера на первом классе на тесте 0.69


In [8]:
def balance_df_by_target(df, target_name, method='over'):

    assert method in ['over', 'under', 'tomek', 'smote'], 'Неверный метод сэмплирования'
    
    target_counts = df[target_name].value_counts()

    major_class_name = target_counts.argmax()
    minor_class_name = target_counts.argmin()

    disbalance_coeff = int(target_counts[major_class_name] / target_counts[minor_class_name]) - 1
    if method == 'over':
        for i in range(disbalance_coeff):
            sample = df[df[target_name] == minor_class_name].sample(target_counts[minor_class_name])
            df = df.append(sample, ignore_index=True)
            
    elif method == 'under':
        df_ = df.copy()
        df = df_[df_[target_name] == minor_class_name]
        tmp = df_[df_[target_name] == major_class_name]
        df = df.append(tmp.iloc[
            np.random.randint(0, tmp.shape[0], target_counts[minor_class_name])
        ], ignore_index=True)

    elif method == 'tomek':
        from imblearn.under_sampling import TomekLinks
        tl = TomekLinks()
        X_tomek, y_tomek = tl.fit_sample(df.drop(columns=target_name), df[target_name])
        df = pd.concat([X_tomek, y_tomek], axis=1)
    
    elif method == 'smote':
        from imblearn.over_sampling import SMOTE
        smote = SMOTE()
        X_smote, y_smote = smote.fit_sample(df.drop(columns=target_name), df[target_name])
        df = pd.concat([X_smote, y_smote], axis=1)

    return df.sample(frac=1) 

_Oversampling_

In [9]:
train_df = pd.concat([pd.DataFrame(X_train), y_train], axis=1)
print(f'До oversampling:\n{y_train.value_counts()}')
over_df = balance_df_by_target(train_df, target_name=target, method='over')
print(f'\nПосле oversampling:\n{over_df[target].value_counts()}')

До oversampling:
0    5586
1    2414
Name: target, dtype: int64

После oversampling:
0    5586
1    4828
Name: target, dtype: int64


In [10]:
lr = LogisticRegressionCV()
lr.fit(over_df.drop(columns=target), over_df[target])

pred_train = lr.predict(over_df.drop(columns=target))
pred_test = lr.predict(X_test)

print(f'F1-мера на первом классе на трейне {round(f1_score(over_df[target], pred_train), 2)}')
print(f'F1-мера на первом классе на тесте {round(f1_score(y_test, pred_test), 2)}')

F1-мера на первом классе на трейне 0.85
F1-мера на первом классе на тесте 0.77


_Undersampling_

In [11]:
print(f'До undersampling:\n{train_df[target].value_counts()}')
under_df = balance_df_by_target(train_df, target_name=target, method='under')
print(f'\nПосле undersampling:\n{under_df[target].value_counts()}')

До undersampling:
0    5586
1    2414
Name: target, dtype: int64

После undersampling:
1    2414
0    2414
Name: target, dtype: int64


In [12]:
lr = LogisticRegressionCV()
lr.fit(under_df.drop(columns=target), under_df[target])

pred_train = lr.predict(under_df.drop(columns=target))
pred_test = lr.predict(X_test)

print(f'F1-мера на первом классе на трейне {round(f1_score(under_df[target], pred_train), 2)}')
print(f'F1-мера на первом классе на тесте {round(f1_score(y_test, pred_test), 2)}')

F1-мера на первом классе на трейне 0.88
F1-мера на первом классе на тесте 0.78


_Undersampling (TomekLinks)_ 

In [13]:
print(f'До undersampling:\n{train_df[target].value_counts()}')
under_df = balance_df_by_target(train_df, target_name=target, method='tomek')
print(f'\nПосле undersampling:\n{under_df[target].value_counts()}')

До undersampling:
0    5586
1    2414
Name: target, dtype: int64

После undersampling:
0    5462
1    2414
Name: target, dtype: int64


In [14]:
lr = LogisticRegressionCV()
lr.fit(under_df.drop(columns=target), under_df[target])

pred_train = lr.predict(under_df.drop(columns=target))
pred_test = lr.predict(X_test)

print(f'F1-мера на первом классе на трейне {round(f1_score(under_df[target], pred_train), 2)}')
print(f'F1-мера на первом классе на тесте {round(f1_score(y_test, pred_test), 2)}')

F1-мера на первом классе на трейне 0.71
F1-мера на первом классе на тесте 0.7


_Oversampling (SMOTE)_

In [15]:
print(f'До undersampling:\n{train_df[target].value_counts()}')
over_df = balance_df_by_target(train_df, target_name=target, method='smote')
print(f'\nПосле oversampling:\n{over_df[target].value_counts()}')

До undersampling:
0    5586
1    2414
Name: target, dtype: int64

После oversampling:
1    5586
0    5586
Name: target, dtype: int64


In [16]:
lr = LogisticRegressionCV()
lr.fit(over_df.drop(columns=target), over_df[target])

pred_train = lr.predict(over_df.drop(columns=target))
pred_test = lr.predict(X_test)

print(f'F1-мера на первом классе на трейне {round(f1_score(over_df[target], pred_train), 2)}')
print(f'F1-мера на первом классе на тесте {round(f1_score(y_test, pred_test), 2)}')

F1-мера на первом классе на трейне 0.87
F1-мера на первом классе на тесте 0.78


_Балансировка через атрибуты моделей (веса классов)_

In [17]:
disbalance

2.3140016570008286

In [18]:
lr = LogisticRegressionCV(
#                             class_weight={0: 1, 1: disbalance},
                            class_weight="balanced"
                            )
lr.fit(X_train, y_train)

pred_train = lr.predict(X_train)
pred_test = lr.predict(X_test)

print(f'F1-мера на первом классе на трейне {round(f1_score(y_train, pred_train), 2)}')
print(f'F1-мера на первом классе на тесте {round(f1_score(y_test, pred_test), 2)}')

F1-мера на первом классе на трейне 0.77
F1-мера на первом классе на тесте 0.78
